# CycleGAN 极简交互演示 (调用官方源码版)

这个版本不再手动定义复杂的网络结构，而是直接**导入**官方源码里的类。
这就好比：我们不再自己造发动机，而是直接去仓库里通过 `import` 拿现成的来用。

### 前置条件
确保你的文件结构如下：
- 本文件 `gradioui_simple.ipynb`
- 目录 `官方源码/pytorch-CycleGAN-and-pix2pix-master/` (官方代码解压在这里)

In [10]:
# 1. 魔法咒语：把官方源码目录加入到 Python 搜索路径
import sys
import os

# 这里的路径就是你存放官方源码的地方
source_code_path = r"官方源码/pytorch-CycleGAN-and-pix2pix-master"
if source_code_path not in sys.path:
    sys.path.append(source_code_path)

print("✅ 已成功连接到官方源码库！")

✅ 已成功连接到官方源码库！


In [11]:
# 2. 直接从官方源码里 Import 我们需要的零件
import torch
import gradio as gr
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

#看好了，这一行就是精华！直接导入，不用自己写几百行代码
from models.networks import ResnetGenerator

In [12]:
# 3. 极其精简的推理引擎
class SimpleInference:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # 直接实例化官方定义的生成器
        # 参数解释：3入3出，64个基础过滤器，不用dropout，9个残差块
        self.net_horse2zebra = ResnetGenerator(3, 3, 64, use_dropout=False, n_blocks=9).to(self.device)
        self.net_apple2orange = ResnetGenerator(3, 3, 64, use_dropout=False, n_blocks=9).to(self.device)
        
        self.load_model(self.net_horse2zebra, "model/horse2zebra.pth")
        self.load_model(self.net_apple2orange, "model/apple2orange.pth")
        
        # 图片预处理流水线
        self.preprocess = transforms.Compose([
            transforms.Resize((256, 256), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def load_model(self, model, path):
        # 加载权重，处理一点脏数据
        state_dict = torch.load(path, map_location=self.device)
        # 官方代码里可能有一些冗余的 key，清理一下
        if hasattr(state_dict, '_metadata'): del state_dict._metadata
        model.load_state_dict(state_dict, strict=False)
        model.eval()
        print(f"🔹 模型已加载: {path}")

    def predict(self, img, mode):
        if img is None: return None
        
        # 1. 选模型
        model = self.net_horse2zebra if "马" in mode else self.net_apple2orange
        
        # 2. 变数据 (Image -> Tensor)
        data = self.preprocess(img).unsqueeze(0).to(self.device)
        
        # 3. 跑模型 (Inference)
        with torch.no_grad():
            output = model(data)
            
        # 4. 变回图 (Tensor -> Image)
        output = (output.squeeze(0).cpu().numpy() + 1) / 2.0 * 255.0
        return output.transpose(1, 2, 0).clip(0, 255).astype(np.uint8)

# 启动引擎
engine = SimpleInference()

🔹 模型已加载: model/horse2zebra.pth
🔹 模型已加载: model/apple2orange.pth


D:\系统缓存\ipykernel_26568\3305020332.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=self.device)


In [13]:
# 4. 界面部分 (完全没变，这就叫前后端分离)
css = """
.fixed-height { height: 350px; }
/* 定义一个橙色按钮的样式 */
.orange-btn { 
    background: #FFB040 !important; 
    color: white !important; 
    border: none !important;
}
.orange-btn:hover { background: #E59400 !important; } /* 鼠标悬停变深 */
"""
with gr.Blocks(css=css) as demo:
    gr.Markdown("## CycleGAN 极简版演示")
    with gr.Row():
        mode = gr.Radio(["马 🐎 → 斑马 🦓", "苹果 🍎 → 橙子 🍊"], value="马 🐎 → 斑马 🦓", label="模式")
    with gr.Row():
        inp = gr.Image(type="pil", label="原图", elem_classes="fixed-height")
        out = gr.Image(type="numpy", label="结果", elem_classes="fixed-height")
    
    # 给按钮加上 elem_classes="orange-btn"
    btn = gr.Button("🚀 转换", elem_classes="orange-btn")
    btn.click(engine.predict, inputs=[inp, mode], outputs=out)

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
